In [1]:
import sys
sys.path.append("..")

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import xgboost
from sklearn.ensemble import RandomForestClassifier
from satreeverify import SATreeAttack

/home/masoud/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
data = load_breast_cancer()
X = data['data']
y = data["target"]
feature_names = data["feature_names"]
y = y==1

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# XGBoost Model

In [5]:
ntrees = 50
clf = xgboost.XGBClassifier(n_estimators=ntrees, max_depth=3).fit(x_train, y_train)
print(clf.score(x_test, y_test))

attack = SATreeAttack(clf)

[22:33:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9912280701754386


/home/masoud/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [6]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.9918581, 0.0081419]], dtype=float32)

In [7]:
result = attack.soft_attack(sample, epsilon)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,13.110000,0.000000,"[-inf, inf]"
1,15.560000,19.545218,0.256119,"[19.5449982, 19.7649994]"
2,87.210000,87.210000,0.000000,"[-inf, inf]"
3,530.200000,530.200000,0.000000,"[-inf, 694.5]"
4,0.139800,0.091573,0.344969,"[0.0899550021, 0.0915749967]"
5,0.176500,0.176500,0.000000,"[0.0713849962, inf]"
6,0.207100,0.207100,0.000000,"[0.100950003, inf]"
7,0.096010,0.065973,0.312850,"[0.0492300019, 0.065990001]"
8,0.192500,0.179620,0.066908,"[-inf, 0.179800004]"
9,0.076920,0.076920,0.000000,"[0.0559, inf]"


In [8]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

In [9]:
result = attack.hard_attack(sample, epsilon, 8)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,13.110000,0.000000,"[-inf, inf]"
1,15.560000,15.519465,0.002605,"[-inf, 15.5349998]"
2,87.210000,87.210000,0.000000,"[-inf, inf]"
3,530.200000,530.200000,0.000000,"[-inf, 696.25]"
4,0.139800,0.099657,0.287147,"[0.0915749967, 0.099665001]"
5,0.176500,0.176500,0.000000,"[0.0713849962, inf]"
6,0.207100,0.207100,0.000000,"[0.100950003, inf]"
7,0.096010,0.065973,0.312850,"[0.0492300019, 0.065990001]"
8,0.192500,0.192500,0.000000,"[-inf, inf]"
9,0.076920,0.055844,0.273998,"[-inf, 0.0559]"


In [10]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

# RandomForest Model

In [11]:
clf = RandomForestClassifier(n_estimators=ntrees, max_depth=3).fit(X, y)
print(clf.score(x_test, y_test))

attack = SATreeAttack(clf)

0.9736842105263158


In [12]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.7316549, 0.2683451]])

In [13]:
result = attack.soft_attack(sample, epsilon)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,10.759229,0.179311,"[9.989500045776367, 10.759999752044678]"
1,15.560000,20.085370,0.290834,"[20.085000038146973, 20.454999923706055]"
2,87.210000,96.620069,0.107901,"[96.61999893188477, 96.69000244140625]"
3,530.200000,530.200000,0.000000,"[-inf, 575.5500183105469]"
4,0.139800,0.090188,0.354877,"[0.07839000225067139, 0.09019999951124191]"
5,0.176500,0.113632,0.356195,"[0.09524999931454659, 0.11365000158548355]"
6,0.207100,0.207100,0.000000,"[0.10459999740123749, inf]"
7,0.096010,0.061535,0.359081,"[0.056165000423789024, 0.06154000014066696]"
8,0.192500,0.184467,0.041729,"[0.15164999663829803, 0.18450000137090683]"
9,0.076920,0.075084,0.023866,"[0.0643249973654747, 0.07509500160813332]"


In [14]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

In [15]:
result = attack.hard_attack(sample, epsilon, 8)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,15.020020,0.145692,"[15.019999980926514, 15.039999961853027]"
1,15.560000,16.455819,0.057572,"[16.454999446868896, 17.27500057220459]"
2,87.210000,97.515789,0.118172,"[97.51499938964844, 98.30500030517578]"
3,530.200000,530.200000,0.000000,"[-inf, 575.5500183105469]"
4,0.139800,0.100618,0.280274,"[0.07839000225067139, 0.10063999891281128]"
5,0.176500,0.144071,0.183734,"[0.11504999920725822, 0.14409999549388885]"
6,0.207100,0.207100,0.000000,"[0.10459999740123749, inf]"
7,0.096010,0.061535,0.359081,"[0.056165000423789024, 0.06154000014066696]"
8,0.192500,0.184316,0.042517,"[-inf, 0.18450000137090683]"
9,0.076920,0.076920,0.000000,"[0.07509500160813332, inf]"


In [16]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))